Note that any backslashes in this notebook may need to be changed to a forward slash depending on the operating system. This cell sets the name of directories where the data is stored.

In [2]:
index = collect(findlast("\\", pwd()))[end]
parent_dir = pwd()[1:index]
cross_section_dir = "C:\\Cross-Section-Data\\"
import_data_dir = parent_dir * "ImportedData\\"
export_data_dir = parent_dir * "ExportedData\\"
endf8_decay_dir = cross_section_dir * "ENDF-B-VIII.0\\decay\\"
jeff33_decay_dir = cross_section_dir * "ENDF_Libraries-2\\JEFF-3.3\\decay\\"
jendl5_decay_dir = cross_section_dir * "jendl5-dec_upd5\\"

"C:\\Cross-Section-Data\\jendl5-dec_upd5\\"

Import packages

In [3]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Plots
using Downloads
#plotlyjs()
P, N, x, Σ, σR, σT, ρ, M, A = symbols("P, N, x, Σ, σ_R, σ_T, ρ, M, A", 
                                        positive = true, real = true)
N = ρ * A/ M
Σ = N * σT  
P = integrate(Σ*exp(-Σ*x), (x, 0, x)) * σR / σT

WebIO._IJuliaInit()

    ⎛     -A⋅x⋅ρ⋅σ_T ⎞
    ⎜     ───────────⎟
    ⎜          M     ⎟
σ_R⋅⎝1 - ℯ           ⎠
──────────────────────
         σ_T          

Now I will find the average decay energies for each nuclide. For the datasets I am using, the average decay energies are all given in keV. 

In [51]:
"248m1Bk"

"248m1Bk"

In [55]:
is_digit(str) = (tryparse(Int64, str) != nothing)
is_digit("1"), is_digit("a")

(true, false)

In [4]:
function get_mass_name(nuclide)
    mass_index = 1
    while index < length(nuclide)-1 && (
            is_digit(nuclide[mass_index]) && ! is_digit(nuclide[mass_index+1]))
    end
    

    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];
function atomic_number(symbol)
    index = 1
    while symbol != element_symbols[index]
        index += 1
        if index > length(element_symbols)
            return "Element not in periodic table"
        end
    end
    return index 
end

atomic_number (generic function with 1 method)

This dictionary stores what strings to look to for in the datasets to find the average energy for each decay type, and for each directory. 

In [5]:
dir_search_string_dict = Dict([
    jeff33_decay_dir => Dict(["alpha" => "ALPHA ENERGY                   =",
    "beta-minus" => "MEAN BETA- ENERGY              =","gamma" => "MEAN GAMMA ENERGY              =", 
    "x-ray" => "MEAN X-RAY ENERGY              =", "auger" => "MEAN AUGER ELECTRON ENERGY     ="
    ]),

    endf8_decay_dir => Dict(["alpha" => "Mean Alpha Energy:", 
    "beta-minus" => "Mean B- Energy:", "beta-plus" => "Mean B+ Energy:",
    "gamma" => "Mean Gamma Energy:", "auger" => "Mean CE+Auger Energy:",
    "neutron" => "Mean Neutron Energy:", "proton" => "Mean Neutron Energy:",
    "half life" => "Parent half-life:"]),

    jendl5_decay_dir => Dict(["alpha" => "Mean Alpha Energy",
    "beta-minus" => "Mean B- Energy", "gamma" => "Mean Gamma Energy",
    "neutron" => "Mean Neutron Energy", "proton" => "Mean Proton Energy",
    "auger" => "Mean CE+Auger Energy", 
    "beta-plus and electron capture" => "Mean B+/EC Energy", "half life" => "T1/2:"])  
])

Dict{String, Dict{String, String}} with 3 entries:
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy", "beta-plus…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"MEAN AUGER ELECTRON ENERGY     ="…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy:", "proton"=…

In [6]:
file_type(file) = file[collect(findlast(".", file))[1]:end]
file_type("data.endf")

".endf"

In [7]:
function get_decay_energy_from_file(file_as_arr, search_string)
    line = [l for l in file_as_arr if length(l) > length(search_string)
             && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))
        return tryparse(Float64, join(split(line, " ", keepempty = false)[1]))
    else 
        return "Energy not found"
    end
end

get_decay_energy_from_file (generic function with 1 method)

In [8]:
is_0_padded(str) = (length(string(parse(Int64, str))) != length(str))
is_0_padded("011"), is_0_padded("110")

(true, false)

find_decay_info is a function that can retrieve requested information on the decay of a given nuclide in either the JENDL5 or ENDF8 directories.

In [9]:
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
            'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
            'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
            'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 
            'w', 'x', 'y', 'z']
function find_decay_info(nuclide, info, dir)
    all_files = Vector(readdir(dir))
    a, el = get_mass_name(nuclide)
    el = string(el)
    z = [i for i in 1:length(element_symbols) if element_symbols[i] == el][1]
    seperator = '_'
    similar_files = [file for file in all_files 
                    if occursin(seperator * el * seperator, file)]
    if length(similar_files) == 0
        seperator = '-'
        similar_files = [file for file in all_files 
                        if occursin(seperator * el * seperator, file)]
    end
    similar_file = similar_files[1]
    first_seperator_index = 1
    while (!(similar_file[first_seperator_index] == seperator) 
            && first_seperator_index < length(similar_file))
        first_seperator_index += 1
    end
    if length(similar_file) == first_seperator_index
        seperator = ""
    end
    digits = [similar_file[x] for x in 1:first_seperator_index if similar_file[x] 
                in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']]
    if is_0_padded(join(digits))
        file_name = (similar_file[1:first_seperator_index] * el * seperator * 
                    lpad(string(a), 3, "0") * file_type(all_files[end-1]))
    else
        file_name = (similar_file[1:first_seperator_index] * el * seperator * 
                    string(a) * file_type(all_files[end-1]))
    end
    if !(file_name in all_files)
        return "insufficient data"
    end
    file_path = dir * file_name
    file_as_array = split(open(f->read(f, String), file_path), "\n")
    search_string = dir_search_string_dict[dir][info]
    line = [l for l in file_as_array if length(l) > length(search_string)
            && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))        
        if dir == jendl5_decay_dir
            unit = split(line, " ", keepempty = false)[4]
        elseif dir == endf8_decay_dir
            unit = split(line, " ", keepempty = false)[2]
        end
        value = tryparse(Float64, join(split(line, " ", keepempty = false)[1]))
        if value == nothing
            return ["STABLE", "STABLE"]
        end
        return value, unit
    else 
        return info * " not found"
    end
end
find_decay_info("63Ni", "half life", jendl5_decay_dir)

(101.27, "y")

In [10]:
find_decay_info("63Ni", "half life", endf8_decay_dir)

(101.2, "Y")

This function is meant for particularly short half lives reported in terms of the decay width. 

In [11]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))
joules_per_MeV = 1000 * joules_per_keV
h = 6.62607015 * 10.0^(-34) / (2 * pi)
half_life_sec(gamma) = h / (joules_per_MeV * gamma)
half_life_sec(4.6)

1.4308955585889278e-22

Very short half lives are often reported in terms of gamma. Source: 

https://www.nndc.bnl.gov/nudat3/help/glossary.jsp#halflife

In [12]:
small_decay_time(nuclide, dir) = half_life_sec(find_decay_info(nuclide, 
                                        "half life", dir)[1])   
small_decay_time("4H", endf8_decay_dir), small_decay_time("4H", jendl5_decay_dir)

(1.4308955585889278e-22, 1.4308955585889278e-22)

In [13]:
decay_or_emission_type_dict = Dict([
    "B-" =>  "beta minus decay", "A" =>  "alpha decay", "B+" =>  "beta plus decay",
    "EC" => "auger and conversion electron"
])

Dict{String, String} with 4 entries:
  "A"  => "alpha decay"
  "B+" => "beta plus decay"
  "EC" => "auger and conversion electron"
  "B-" => "beta minus decay"

In [14]:
function get_metastable_file_paths(dir, file_type)
    l = length(file_type)
    decay_files = [x for x in readdir(dir) 
                    if length(x) > l && x[end-(l-1):end] == file_type]
    metastable_decay_files = [x for x in decay_files if occursin("m", x)]
    return [dir * file for file in metastable_decay_files]
end
function get_metastable_file_paths(dir)
    example_path = readdir(dir)[end-1]
    file_type_index = collect(findlast(".", example_path))[1]
    file_type = example_path[file_type_index:end]
    return get_metastable_file_paths(dir, file_type)
end
endf8_metstable_paths = get_metastable_file_paths(endf8_decay_dir)

906-element Vector{String}:
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-011_Na_024m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-013_Al_024m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-013_Al_026m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-017_Cl_034m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-017_Cl_038m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-019_K_038m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_042m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_044m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_045m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_046m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_050m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-023_V_044m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-023_V_046m1.endf"
 ⋮
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\d

In [15]:
jendl5_metastable_paths = get_metastable_file_paths(jendl5_decay_dir)

971-element Vector{String}:
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-242.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-243.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-244.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-245.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-246.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-247.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-247m1.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-248.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-249.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-250.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-250m1.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-251.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-252.dat"
 ⋮
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\96-Cm-245.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\96-Cm-246.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\96-Cm-247.dat"
 "C:\\Cross-Section-Data

This function returns the name of an isotope given its file path in the ENDF8 directory. The last five characters in this directory are ".endf" and the first 8 characters are: "dec-", atomic number, and "_". Note that this is note the case in the ENDF-Libraries-2 (that version of ENDF8 decay data is not being used here because it is missing some metastabale isotopes).

Additionally, the file names in JENDL5 are not zero-padded, so this function accounts for that.

In [16]:
function isotope_name(file_path) 
    file = split(file_path, "\\")[end]
    seperation_characters = ['-', '_']
    relevant_indices = [i for i in 1:length(file) 
                        if !(file[i] in seperation_characters)]
    file = file[relevant_indices]
    period_index = 1
    while file[period_index] != '.' && period_index < length(file)
        period_index += 1
    end
    file = string(split(file[1:period_index-1], "dec")[end])
    symbol_index = 1
    while !(file[symbol_index] in alphabet) && symbol_index < length(file)
        symbol_index += 1
    end
    file = file[symbol_index:end]
    number_index = 1
    while (file[number_index] in alphabet) && number_index < length(file)
        number_index += 1
    end
    return file[number_index:end] * file[1:number_index-1]
end
isotope_name("C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-110_Ds_279m1.endf")

"279m1Ds"

In [17]:
isotope_name("C:\\Cross-Section-Data\\jendl5-dec_upd5\\97-Bk-248m1.dat")

"248m1Bk"

This function will take a vector of strings as an input. Then it will find the strings that containt a given substring. Then it will return those strings without the given substring.

In [18]:
filter_substring(vector_of_strings, substring) = [replace(vector_of_strings[i], substring => "") 
    for i in 1:length(vector_of_strings) if occursin(substring, vector_of_strings[i])]

filter_substring (generic function with 1 method)

Here m_isotope denotes a given metastable isotope. 
I will make it return the file names of all the ground states of the metastable isotopes in a given directory.

In [19]:
function ground_state_files(dir)
    all_files = Vector(readdir(dir))
    example_path = all_files[end-1] 
    file_type_index = collect(findlast(".", example_path))[1]  
    file_type = example_path[file_type_index:end]
    l = length(file_type)
    file_paths = [file_path for file_path in all_files
                    if length(file_path) > l && file_path[end-(l-1):end] == file_type]
    files = [filter_substring(file_paths, "m1"); filter_substring(file_paths, "m2");
            filter_substring(file_paths, "m3");]
    return [dir * file for file in files if file in all_files]
    end
ground_state_files_endf8 = ground_state_files(endf8_decay_dir)

737-element Vector{String}:
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-011_Na_024.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-013_Al_024.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-013_Al_026.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-017_Cl_034.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-017_Cl_038.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-019_K_038.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_042.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_044.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_045.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_046.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_050.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-023_V_044.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-023_V_046.endf"
 ⋮
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-083_Bi_196.endf"
 "C:\\

In [20]:
print("C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_045.endf")


C:\Cross-Section-Data\ENDF-B-VIII.0\decay\dec-021_Sc_045.endf

In [21]:
ground_state_files_jendl5 = ground_state_files(jendl5_decay_dir)

797-element Vector{String}:
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-247.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-250.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\101-Md-245.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\101-Md-247.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\101-Md-249.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\101-Md-254.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\101-Md-258.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\102-No-251.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\102-No-254.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\103-Lr-253.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\103-Lr-255.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\104-Rf-257.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\104-Rf-261.dat"
 ⋮
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\83-Bi-200.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\83-Bi-204.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\83-Bi-212.dat"
 "C:\\Cross-Section-Data\\je

Now we find the half liives of the ground states

In [22]:
function ground_state_half_lives(dir)
    ground_state_file_paths = ground_state_files(dir)
    half_life_with_units = [find_decay_info(isotope_name(file), "half life", dir) 
                            for file in ground_state_file_paths]
    return half_life_with_units
end
ground_state_half_lives_endf8 = ground_state_half_lives(endf8_decay_dir)

737-element Vector{Any}:
 (14.997, "H")
 (2.053, "S")
 (717000.0, "Y")
 (1.5264, "S")
 (37.23, "M")
 (7.636, "M")
 (680.8, "MS")
 (3.97, "H")
 ["STABLE", "STABLE"]
 (83.79, "D")
 (102.5, "S")
 (111.0, "MS")
 (422.5, "MS")
 ⋮
 (308.0, "S")
 (10.3, "M")
 (36.4, "M")
 (11.22, "H")
 (60.55, "M")
 (1.74, "H")
 (43.0, "S")
 (184.0, "S")
 (1.7, "S")
 (16.0, "S")
 (16.02, "H")
 (2.0, "S")

In [23]:
units = [x[2] for x in ground_state_half_lives_endf8]
unique(units)

9-element Vector{AbstractString}:
 "H"
 "S"
 "Y"
 "M"
 "MS"
 "STABLE"
 "D"
 "US"
 "ms"

In [24]:
for i in 1:length(units) 
    if units[i] == "US"
        println(ground_state_files_endf8[i])
    end
end

C:\Cross-Section-Data\ENDF-B-VIII.0\decay\dec-079_Au_170.endf
C:\Cross-Section-Data\ENDF-B-VIII.0\decay\dec-079_Au_171.endf
C:\Cross-Section-Data\ENDF-B-VIII.0\decay\dec-084_Po_212.endf
C:\Cross-Section-Data\ENDF-B-VIII.0\decay\dec-089_Ac_216.endf


In [25]:
ground_state_half_lives_jendl5 = ground_state_half_lives(jendl5_decay_dir)

797-element Vector{Any}:
 (31.0, "s")
 (30.0, "m")
 "half life not found"
 (1.2, "s")
 (23.8, "s")
 (10.0, "m")
 (51.5, "d")
 (0.8, "s")
 (51.0, "s")
 (0.64, "s")
 (31.1, "s")
 (4.4, "s")
 (70.0, "s")
 ⋮
 (36.4, "m")
 (11.22, "h")
 (1.01, "h")
 (1.74, "h")
 (28.0, "ms")
 (43.0, "s")
 (184.0, "s")
 (1.9, "s")
 (16.0, "s")
 (16.02, "h")
 (1.74, "s")
 (0.609, "s")

In [26]:
units = [x[2] for x in ground_state_half_lives_jendl5]
unique(units)

7-element Vector{Any}:
 "s"
 "m"
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 "d"
 "ms"
 "h"
 "y"

In [27]:
for i in 1:length(units) 
    if units[i] == 'a'
        println(ground_state_files_jendl5[i])
    end
end

C:\Cross-Section-Data\jendl5-dec_upd5\101-Md-245.dat
C:\Cross-Section-Data\jendl5-dec_upd5\21-Sc-45.dat
C:\Cross-Section-Data\jendl5-dec_upd5\32-Ge-73.dat
C:\Cross-Section-Data\jendl5-dec_upd5\33-As-75.dat
C:\Cross-Section-Data\jendl5-dec_upd5\34-Se-77.dat
C:\Cross-Section-Data\jendl5-dec_upd5\35-Br-79.dat
C:\Cross-Section-Data\jendl5-dec_upd5\36-Kr-83.dat
C:\Cross-Section-Data\jendl5-dec_upd5\38-Sr-87.dat
C:\Cross-Section-Data\jendl5-dec_upd5\39-Y-89.dat
C:\Cross-Section-Data\jendl5-dec_upd5\40-Zr-90.dat
C:\Cross-Section-Data\jendl5-dec_upd5\41-Nb-93.dat
C:\Cross-Section-Data\jendl5-dec_upd5\45-Rh-103.dat
C:\Cross-Section-Data\jendl5-dec_upd5\45-Rh-90.dat
C:\Cross-Section-Data\jendl5-dec_upd5\46-Pd-95.dat
C:\Cross-Section-Data\jendl5-dec_upd5\47-Ag-107.dat
C:\Cross-Section-Data\jendl5-dec_upd5\47-Ag-109.dat
C:\Cross-Section-Data\jendl5-dec_upd5\47-Ag-94.dat
C:\Cross-Section-Data\jendl5-dec_upd5\48-Cd-111.dat
C:\Cross-Section-Data\jendl5-dec_upd5\49-In-113.dat
C:\Cross-Section-Data\jen

I think all of the above files are in ENDF8. 

In [28]:
endf8_decay_dir

"C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\"

C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-083_Bi_196.endf

In [29]:
time_unit = Dict(["Y" => 365.25 * 24 * 3600, "y" => 365.25 * 24 * 3600, 
    "m" => 60, "M" => 60, "h" => 3600, "H" => 3600, "US" => .000001, 
    "D" => 24 * 3600, "d" => 24 * 3600, "s" => 1, "S" => 1, "MS" => .001,
    "ms" => .001])
half_life_sec(value, unit) = value * time_unit[unit]

half_life_sec (generic function with 2 methods)

In [30]:
ground_state_info_dict = Dict([])
for i in 1:length(ground_state_files_jendl5)
    isotope = isotope_name(ground_state_files_jendl5[i])
    if ground_state_half_lives_jendl5[i][2] == 'a'
        a, el = get_mass_name(isotope)
        singular_half_life_with_unit = find_decay_info(isotope, "half life", endf8_decay_dir)
        if singular_half_life_with_unit[1] == "STABLE" 
            ground_state_info_dict[isotope] = "STABLE"
        else
            ground_state_info_dict[isotope] = half_life_sec(
                singular_half_life_with_unit[1], singular_half_life_with_unit[2])
        end
    else
        if ground_state_half_lives_jendl5[i][1] == "STABLE"
            ground_state_info_dict[isotope] = "STABLE"
        else
            ground_state_info_dict[isotope] = half_life_sec(
                ground_state_half_lives_jendl5[i][1], ground_state_half_lives_jendl5[i][2])
        end
    end
end
ground_state_info_dict
    

Dict{Any, Any} with 717 entries:
  "113Sn" => 9.94378e6
  "167Re" => 3.4
  "174Ir" => 7.9
  "90Rh"  => 0.012
  "118In" => 5.0
  "188Bi" => 0.265
  "129La" => 696.0
  "87Zr"  => 6048.0
  "108In" => 3480.0
  "143Gd" => 39.0
  "189Bi" => 0.674
  "129Cd" => 0.151
  "102Nb" => 4.3
  "254Es" => 2.38205e7
  "189Au" => 1722.0
  "151Tb" => 63396.0
  "129Nd" => 6.7
  "138Pm" => 10.0
  "76Br"  => 58320.0
  "242Np" => 132.0
  "113Ru" => 0.8
  "194Ir" => 69408.0
  "136Ba" => "STABLE"
  "185Au" => 255.0
  "146Tm" => 0.068
  ⋮       => ⋮

In [31]:
ground_state_half_lives_dict = Dict([])
stable_ground_states, unstable_ground_states, half_lives = [], [], Float64[]
for key in keys(ground_state_info_dict)
    if ground_state_info_dict[key] != "STABLE" 
        push!(unstable_ground_states, key)
        push!(half_lives, ground_state_info_dict[key])
    else
        push!(stable_ground_states, key)
    end
end

In [32]:
PlotlyJS.plot(half_lives, Layout(yaxis_type = "log",
            title = "Half-life in Seconds for the Ground States of Nuclear Isomers"), 
            mode = "markers", text = unstable_ground_states)

data: [
  "scatter with fields mode, text, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

Now to find the ground states with half-lives longer than 4 months

In [33]:
half_lives_dict = Dict(["Half_life" => half_lives, 
                    "Isotope" => unstable_ground_states])
half_lives_df = DataFrame(half_lives_dict)

,Half_life,Isotope
,Any,Any
1,9.94378e6,113Sn
2,3.4,167Re
3,7.9,174Ir
4,0.012,90Rh
5,5.0,118In
6,0.265,188Bi
7,696.0,129La
8,6048.0,87Zr
9,3480.0,108In


In [49]:
function get_metastable_file_paths(dir, file_type, energy_levels)
    l = length(file_type)
    decay_files = [x for x in readdir(dir) 
                    if length(x) > l && x[end-(l-1):end] == file_type]
    search_term = "m" * string(energy_levels)
    metastable_decay_files = [x for x in decay_files if occursin(search_term , x)]
    return [dir * file for file in metastable_decay_files]
end
function metastable_half_lives(dir, energy_levels)
    all_files = Vector(readdir(dir))
    example_path = all_files[end-1] 
    file_type_index = collect(findlast(".", example_path))[1]  
    file_type = example_path[file_type_index:end]
    file_paths = get_metastable_file_paths(dir, file_type, energy_levels)
    search_term = "m" * string(energy_levels)
    ground_state_paths = filter_substring(file_paths, search_term)
    println(ground_state_paths)
    half_life_with_units = [find_decay_info(isotope_name(file), "half life", dir) 
                            for file in file_paths]
    return half_life_with_units, file_paths
end
output = metastable_half_lives(jendl5_decay_dir, 2)
metastable_half_lives_jendl5, metastable_files_jendl5 = output

["C:\\Cross-Section-Data\\jendl5-dec_upd5\\103-Lr-255.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\29-Cu-70.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\30-Zn-61.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\30-Zn-73.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\39-Y-88.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\39-Y-97.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\41-Nb-90.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\45-Rh-92.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\47-Ag-116.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\47-Ag-94.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\47-Ag-95.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\49-In-109.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\49-In-114.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\49-In-116.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\49-In-118.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\49-In-120.dat", "C:\\Cross-Section-Data\\jendl5-dec_upd5\\49-In-122.dat", "C:\\Cross-Section-Data

LoadError: BoundsError: attempt to access 0-element Vector{Int64} at index [1]

In [42]:
metastable_info_dict = Dict([])
for i in 1:length(metastable_files_jendl5)
    isotope = isotope_name(metastable_files_jendl5[i])
    if metastable_half_lives_jendl5[i][2] == 'a'
        a, el = get_mass_name(isotope)
        singular_half_life_with_unit = find_decay_info(isotope, "half life", endf8_decay_dir)
        if singular_half_life_with_unit[1] == "STABLE" 
            metastable_info_dict[isotope] = "STABLE"
        else
            metastable_info_dict[isotope] = half_life_sec(
                singular_half_life_with_unit[1], singular_half_life_with_unit[2])
        end
    else
        if metastable_half_lives_jendl5[i][1] == "STABLE"
            metastable_info_dict[isotope] = "STABLE"
        else
            metastable_info_dict[isotope] = half_life_sec(
                metastable_half_lives_jendl5[i][1], metastable_half_lives_jendl5[i][2])
        end
    end
end
metastable_info_dict

Dict{Any, Any} with 78 entries:
  "94Ag"  => 0.035
  "127In" => 1.09
  "155Lu" => 0.068
  "120In" => 3.08
  "124Sb" => 5.20128e6
  "206Fr" => 16.0
  "188Tl" => 71.0
  "182Ta" => 9.91354e6
  "160Ho" => 1536.0
  "131Te" => 1500.0
  "158Ho" => 678.0
  "179Hf" => "STABLE"
  "194Bi" => 95.0
  "196Bi" => 307.8
  "97Y"   => 3.74
  "118In" => 5.0
  "179Ta" => 5.74348e7
  "152Pm" => 247.2
  "125In" => 2.36
  "154Tb" => 77400.0
  "70Cu"  => 44.5
  "192Au" => 17784.0
  "191Ir" => "STABLE"
  "162Lu" => 82.2
  "204Fr" => 1.9
  ⋮       => ⋮

In [35]:
all_ground_states = [[half_lives_df[row,"Isotope"] for row in 1:size(half_lives_df)[1]
    if half_lives_df[row,"Half_life"] > 10^7]; stable_ground_states]
two_energy_level_isotopes = [get_mass_name(isotope_name(file)) 
                            for file in readdir(jendl5_decay_dir) if occursin("m2", file)]

78-element Vector{Tuple{Int64, String}}:
 (255, "M2")
 (70, "M2")
 (61, "M2")
 (73, "M2")
 (88, "M2")
 (97, "M2")
 (90, "M2")
 (92, "M2")
 (116, "M2")
 (94, "M2")
 (95, "M2")
 (109, "M2")
 (114, "M2")
 ⋮
 (196, "M2")
 (198, "M2")
 (200, "M2")
 (204, "M2")
 (212, "M2")
 (205, "M2")
 (193, "M2")
 (200, "M2")
 (202, "M2")
 (204, "M2")
 (206, "M2")
 (242, "M2")